# Project for Week3 Q3

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

## Loading the dataset

In [8]:
neighborhoods = pd.read_csv('location.csv').drop('Unnamed: 0',axis=1)

In [20]:
neighborhoods.head()

Postal code      Borough                              Neighborhood  \
0         M1B  Scarborough                           Malvern , Rouge   
1         M1C  Scarborough  Rouge Hill , Port Union , Highland Creek   
2         M1E  Scarborough       Guildwood , Morningside , West Hill   
3         M1G  Scarborough                                    Woburn   
4         M1H  Scarborough                                 Cedarbrae   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476

## Explore Neighborhoods in Toronto

In [6]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
# create map of New York using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

In [15]:
CLIENT_ID = 'QNIHSMJKWAP2RAMB2IJQL25AW0NEQXUYMDUNAPRT2XGTVYIJ' # your Foursquare ID
CLIENT_SECRET = 'YDBV5IBV4CJBVH5Z23XIXI5UX1SRYFE5BNAFEQ30SF4QZWNS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
tor_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Malvern , Rouge
Rouge Hill , Port Union , Highland Creek
Guildwood , Morningside , West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park , Ionview , East Birchmount Park
Golden Mile , Clairlea , Oakridge
Cliffside , Cliffcrest , Scarborough Village West
Birch Cliff , Cliffside West
Dorset Park , Wexford Heights , Scarborough Town Centre
Wexford , Maryvale
Agincourt
Clarks Corners , Tam O'Shanter , Sullivan
Milliken , Agincourt North , Steeles East , L'Amoreaux East
Steeles West , L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview , Henry Farm , Oriole
Bayview Village
York Mills , Silver Hills
Willowdale , Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor , Wilson Heights , Downsview North
Northwood Park , York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill , Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West , Riverdale
India Bazaar , The Beaches 

In [22]:
print(tor_venues.shape)
tor_venues.head()

(2192, 7)


Neighborhood  Neighborhood Latitude  \
0                           Malvern , Rouge              43.806686   
1                           Malvern , Rouge              43.806686   
2  Rouge Hill , Port Union , Highland Creek              43.784535   
3  Rouge Hill , Port Union , Highland Creek              43.784535   
4  Rouge Hill , Port Union , Highland Creek              43.784535   

   Neighborhood Longitude                                              Venue  \
0              -79.194353                                            Wendy’s   
1              -79.194353                       T Hamilton & Son Roofing Inc   
2              -79.160497                              Royal Canadian Legion   
3              -79.160497                     Scarborough Historical Society   
4              -79.160497  SEBS Engineering Inc. (Sustainable Energy and ...   

   Venue Latitude  Venue Longitude              Venue Category  
0       43.807448       -79.199056        Fast Food Restaurant  
1       43.807985       -79.198194  Construction & Landscaping  
2       43.782533       -79.163085                         Bar  
3       43.788755       -79.162438              History Museum  
4       43.782371       -79.156820  Construction & Landscaping

In [23]:
tor_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Agincourt                                                               4   
Alderwood , Long Branch                                                10   
Bathurst Manor , Wilson Heights , Downsview North                      22   
Bayview Village                                                         4   
Bedford Park , Lawrence Manor East                                     23   
Berczy Park                                                            56   
Birch Cliff , Cliffside West                                            4   
Brockton , Parkdale Village , Exhibition Place                         22   
Business reply mail Processing CentrE                                  16   
CN Tower , King and Spadina , Railway Lands , H...                     18   
Caledonia-Fairbanks                                                     4   
Canada Post Gateway Processing Centre                                  13   
Cedarbrae                                                               8   
Central Bay Street                                                     75   
Christie                                                               18   
Church and Wellesley                                                   79   
Clarks Corners , Tam O'Shanter , Sullivan                              14   
Cliffside , Cliffcrest , Scarborough Village West                       3   
Commerce Court , Victoria Hotel                                       100   
Davisville                                                             34   
Davisville North                                                        7   
Del Ray , Mount Dennis , Keelsdale and Silverthorn                      5   
Don Mills                                                              24   
Dorset Park , Wexford Heights , Scarborough Tow...                      6   
Downsview                                                              14   
Dufferin , Dovercourt Village                                          16   
East Toronto                                                            3   
Eringate , Bloordale Gardens , Old Burnhamthorp...                      7   
Fairview , Henry Farm , Oriole                                         64   
First Canadian Place , Underground city                               100   
Forest Hill North & West                                                5   
Garden District, Ryerson                                              100   
Glencairn                                                               5   
Golden Mile , Clairlea , Oakridge                                      10   
Guildwood , Morningside , West Hill                                     7   
Harbourfront East , Union Station , Toronto Isl...                    100   
High Park , The Junction South                                         24   
Hillcrest Village                                                       4   
Humber Summit                                                           2   
Humberlea , Emery                                                       1   
Humewood-Cedarvale                                                      4   
India Bazaar , The Beaches West                                        20   
Kennedy Park , Ionview , East Birchmount Park                           7   
Kensington Market , Chinatown , Grange Park                            74   
Kingsview Village , St. Phillips , Martin Grove...                      4   
Lawrence Manor , Lawrence Heights                                      11   
Lawrence Park                                                           3   
Leaside                                                                33   
Little Portugal , Trinity                                              50   
Malvern , Rouge                                                         2   
Milliken , Agincourt North , Steeles East , L'A.

In [24]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 270 uniques categories.


## Analyze Each Neighborhood

In [45]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 
tor_onehot = tor_onehot.set_index('Neighborhood').reset_index()

tor_onehot.head()

Neighborhood  Accessories Store  \
0                           Malvern , Rouge                  0   
1                           Malvern , Rouge                  0   
2  Rouge Hill , Port Union , Highland Creek                  0   
3  Rouge Hill , Port Union , Highland Creek                  0   
4  Rouge Hill , Port Union , Highland Creek                  0   

   Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0                  0        0                   0             0   
1                  0        0                   0             0   
2                  0        0                   0             0   
3                  0        0                   0             0   
4                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  \
0             0         0            0           0                    0   
1             0         0            0           0                    0   
2             0         0            0           0                    0   
3             0         0            0           0                    0   
4             0         0            0           0                    0   

   Asian Restaurant  Athletics & Sports  Auto Garage  Auto Workshop  \
0                 0                   0            0              0   
1                 0                   0            0              0   
2                 0                   0            0              0   
3                 0                   0            0              0   
4                 0                   0            0              0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  Baseball Field  \
0          0           0           0       0     0    0               0   
1          0           0           0       0     0    0               0   
2          0           0           0       0     0    1               0   
3          0           0           0       0     0    0               0   
4          0           0           0       0     0    0               0   

   Baseball Stadium  Basketball Court  Basketball Stadium  Beach  \
0                 0                 0                   0      0   
1                 0                 0                   0      0   
2                 0                 0                   0      0   
3                 0                 0                   0      0   
4                 0                 0                   0      0   

   Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  Bike Shop  \
0                0         0           0                   0          0   
1                0         0           0                   0          0   
2                0         0           0                   0          0   
3                0         0           0                   0          0   
4                0         0           0                   0          0   

   Bistro  Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  \
0       0              0          0         0                     0   
1       0              0          0         0                     0   
2       0              0          0         0                     0   
3       0              0          0         0                     0   
4       0              0          0         0                     0   

   Breakfast Spot  Brewery  Bridal Shop  Bubble Tea Shop  Building  \
0               0        0            0                0         0   
1           

In [46]:
tor_onehot.shape

(2192, 270)

In [48]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

Neighborhood  Accessories Store  \
0                                           Agincourt           0.000000   
1                             Alderwood , Long Branch           0.000000   
2   Bathurst Manor , Wilson Heights , Downsview North           0.000000   
3                                     Bayview Village           0.000000   
4                  Bedford Park , Lawrence Manor East           0.000000   
5                                         Berczy Park           0.000000   
6                        Birch Cliff , Cliffside West           0.000000   
7      Brockton , Parkdale Village , Exhibition Place           0.000000   
8               Business reply mail Processing CentrE           0.000000   
9   CN Tower , King and Spadina , Railway Lands , ...           0.000000   
10                                Caledonia-Fairbanks           0.000000   
11              Canada Post Gateway Processing Centre           0.000000   
12                                          Cedarbrae           0.000000   
13                                 Central Bay Street           0.000000   
14                                           Christie           0.000000   
15                               Church and Wellesley           0.000000   
16          Clarks Corners , Tam O'Shanter , Sullivan           0.000000   
17  Cliffside , Cliffcrest , Scarborough Village West           0.000000   
18                    Commerce Court , Victoria Hotel           0.000000   
19                                         Davisville           0.000000   
20                                   Davisville North           0.000000   
21  Del Ray , Mount Dennis , Keelsdale and Silvert...           0.000000   
22                                          Don Mills           0.000000   
23  Dorset Park , Wexford Heights , Scarborough To...           0.000000   
24                                          Downsview           0.000000   
25                      Dufferin , Dovercourt Village           0.000000   
26                                       East Toronto           0.000000   
27  Eringate , Bloordale Gardens , Old Burnhamthor...           0.000000   
28                     Fairview , Henry Farm , Oriole           0.000000   
29            First Canadian Place , Underground city           0.000000   
30                           Forest Hill North & West           0.000000   
31                           Garden District, Ryerson           0.000000   
32                                          Glencairn           0.000000   
33                  Golden Mile , Clairlea , Oakridge           0.000000   
34                Guildwood , Morningside , West Hill           0.000000   
35  Harbourfront East , Union Station , Toronto Is...           0.000000   
36                     High Park , The Junction South           0.000000   
37                                  Hillcrest Village           0.000000   
38                                      Humber Summit           0.000000   
39                                  Humberlea , Emery           0.000000   
40                                 Humewood-Cedarvale           0.000000   
41                    India Bazaar , The Beaches West           0.000000   
42      Kennedy Park , Ionview , East Birchmount Park           0.000000   
43        Kensington Market , Chinatown , Grange Park           0.000000   
44  Kingsview Village , St. Phillips , Martin Grov...           0.000000   
45                  Lawrence Manor , Lawrence Heights           0.090909   
46                                      Lawrence Park           0.000000   
47                                            Leaside           0.000000   
48                          Little Portugal , Trinity           0.000000   
49                                    Malvern , Rouge           0.000000   
50  Milliken , Agincourt North , Steeles East , L'...           0.000000   
51  Mimico NW , The Queensway West , South of Bloo...           0.000000   
52           

In [50]:
tor_grouped.shape

(94, 270)

In [52]:
# First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood  \
0                                          Agincourt   
1                            Alderwood , Long Branch   
2  Bathurst Manor , Wilson Heights , Downsview North   
3                                    Bayview Village   
4                 Bedford Park , Lawrence Manor East   

       1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0  Latin American Restaurant                Lounge          Skating Rink   
1                Pizza Place           Coffee Shop              Pharmacy   
2                Coffee Shop                  Bank             Gift Shop   
3        Japanese Restaurant                  Café                  Bank   
4                 Restaurant           Coffee Shop        Sandwich Place   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0        Breakfast Spot      Doner Restaurant    Dim Sum Restaurant   
1                   Pub                  Pool        Sandwich Place   
2    Frozen Yogurt Shop        Sandwich Place           Bridal Shop   
3    Chinese Restaurant          Dessert Shop                 Diner   
4    Italian Restaurant              Pharmacy     Indian Restaurant   

  7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0                 Diner        Discount Store   Distribution Center   
1          Dance Studio                   Gym          Skating Rink   
2                 Diner            Restaurant         Deli / Bodega   
3        Discount Store   Distribution Center               Dog Run   
4                   Pub      Sushi Restaurant                  Café   

  10th Most Common Venue  
0                Dog Run  
1                Dog Run  
2            Supermarket  
3            Yoga Studio  
4   Fast Food Restaurant

## Cluster Neighborhoods

In [65]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [66]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() # check the last columns!

Postal code      Borough                              Neighborhood  \
0         M1B  Scarborough                           Malvern , Rouge   
1         M1C  Scarborough  Rouge Hill , Port Union , Highland Creek   
2         M1E  Scarborough       Guildwood , Morningside , West Hill   
3         M1G  Scarborough                                    Woburn   
4         M1H  Scarborough                                 Cedarbrae   

    Latitude  Longitude  Cluster Labels       1st Most Common Venue  \
0  43.806686 -79.194353             0.0  Construction & Landscaping   
1  43.784535 -79.160497             4.0                         Bar   
2  43.763573 -79.188711             4.0           Electronics Store   
3  43.770992 -79.216917             4.0                 Coffee Shop   
4  43.773136 -79.239476             4.0         Fried Chicken Joint   

        2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0        Fast Food Restaurant            Donut Shop    Dim Sum Restaurant   
1  Construction & Landscaping        History Museum           Yoga Studio   
2                Intersection   Rental Car Location    Mexican Restaurant   
3            Insurance Office     Korean Restaurant           Yoga Studio   
4                 Gas Station      Hakka Restaurant                Bakery   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                 Diner        Discount Store   Distribution Center   
1            Donut Shop                 Diner        Discount Store   
2                  Bank        Medical Center        Breakfast Spot   
3      Doner Restaurant    Dim Sum Restaurant                 Diner   
4                  Bank    Athletics & Sports       Thai Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0               Dog Run      Doner Restaurant            Yoga Studio  
1   Distribution Center               Dog Run       Doner Restaurant  
2            Donut Shop      Doner Restaurant                Dog Run  
3        Discount Store   Distribution Center                Dog Run  
4  Caribbean Restaurant          Dessert Shop     Dim Sum Restaurant

In [71]:
tor_merged = tor_merged.dropna()

In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

### Cluster 1

In [73]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

Borough  Cluster Labels       1st Most Common Venue  \
0   Scarborough             0.0  Construction & Landscaping   
91    Etobicoke             0.0  Construction & Landscaping   
97   North York             0.0              Baseball Field   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0   Fast Food Restaurant            Donut Shop    Dim Sum Restaurant   
91        Baseball Field           Yoga Studio            Donut Shop   
97           Yoga Studio    Dim Sum Restaurant                 Diner   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                  Diner        Discount Store   Distribution Center   
91                 Diner        Discount Store   Distribution Center   
97        Discount Store   Distribution Center               Dog Run   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                Dog Run      Doner Restaurant            Yoga Studio  
91               Dog Run      Doner Restaurant              Drugstore  
97      Doner Restaurant            Donut Shop     Falafel Restaurant

### Cluster 2

In [74]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
14        Scarborough             1.0                  Park   
23         North York             1.0                  Park   
25         North York             1.0                  Park   
40          East York             1.0                  Park   
44    Central Toronto             1.0                  Park   
48    Central Toronto             1.0                  Park   
50   Downtown Toronto             1.0                  Park   
64    Central Toronto             1.0                  Park   
74               York             1.0                  Park   
79         North York             1.0                  Park   
90          Etobicoke             1.0                  Park   
98               York             1.0                  Park   
100         Etobicoke             1.0                  Park   

          2nd Most Common Venue 3rd Most Common Venue  \
14                   Playground               Dog Run   
23            Convenience Store                  Bank   
25            Food & Drink Shop               Dog Run   
40            Convenience Store           Coffee Shop   
44                     Bus Line           Swim School   
48                  Summer Camp     Electronics Store   
50                        Trail            Playground   
64                        Trail              Bus Line   
74                Women's Store                Market   
79   Construction & Landscaping                Bakery   
90                        River                  Pool   
98                Deli / Bodega   Empanada Restaurant   
100                 Pizza Place        Sandwich Place   

           4th Most Common Venue        5th Most Common Venue  \
14              Department Store                 Dessert Shop   
23            Dim Sum Restaurant                        Diner   
25                  Dessert Shop           Dim Sum Restaurant   
40                  Dessert Shop           Dim Sum Restaurant   
44                       Dog Run           Dim Sum Restaurant   
48   Eastern European Restaurant          Dumpling Restaurant   
50              Department Store                 Dessert Shop   
64              Sushi Restaurant                Jewelry Store   
74                       Dog Run                 Dessert Shop   
79              Basketball Court            Electronics Store   
90                 Deli / Bodega             Department Store   
98             Electronics Store  Eastern European Restaurant   
100            Mobile Phone Shop               Discount Store   

           6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
14            Dim Sum Restaurant                 Diner        Discount Store   
23                Discount Store   Distribution Center               Dog Run   
25                         Diner        Discount Store   Distribution Center   
40                         Diner        Discount Store   Distribution Center   
44                         Diner        Discount Store   Distribution Center   
48                     Drugstore            Donut Shop      Doner Restaurant   
50            Dim Sum Restaurant                 Diner        Discount Store   
64                  Dessert Shop    Dim Sum Restaurant                 Diner   
74            Dim Sum Restaurant                 Diner        Discount Store   
79   Eastern European Restaurant   Dumpling Restaurant   Empanada Restaurant   
90                  Dessert Shop    Dim Sum Restaurant                 Diner   
98           Dumpling Restaurant             Drugstore            Donut Shop   
100                Deli / Bodega      Department Store          Dessert Shop   

    9th Most Common Venue 10th Most Common Venue  
14    Distribution Center       Doner Restaurant  
23       Doner Restaurant       Department Store  
25       Doner Restaurant            Event Space  
40                Dog Run       Doner Restaurant  
44       Doner Restaurant       Department St

### Cluster 3

In [75]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
20  North York             3.0             Cafeteria           Yoga Studio   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
20               Dog Run          Dessert Shop    Dim Sum Restaurant   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
20                 Diner        Discount Store   Distribution Center   

   9th Most Common Venue 10th Most Common Venue  
20      Doner Restaurant          Deli / Bodega

### Cluster 4

In [77]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

Borough  Cluster Labels       1st Most Common Venue  \
1         Scarborough             4.0                         Bar   
2         Scarborough             4.0           Electronics Store   
3         Scarborough             4.0                 Coffee Shop   
4         Scarborough             4.0         Fried Chicken Joint   
5         Scarborough             4.0  Construction & Landscaping   
6         Scarborough             4.0            Department Store   
7         Scarborough             4.0                      Bakery   
8         Scarborough             4.0         American Restaurant   
9         Scarborough             4.0       General Entertainment   
10        Scarborough             4.0           Indian Restaurant   
11        Scarborough             4.0                 Auto Garage   
12        Scarborough             4.0   Latin American Restaurant   
13        Scarborough             4.0                    Pharmacy   
15        Scarborough             4.0                 Coffee Shop   
17         North York             4.0    Mediterranean Restaurant   
18         North York             4.0              Clothing Store   
19         North York             4.0         Japanese Restaurant   
22         North York             4.0                 Coffee Shop   
24         North York             4.0                 Coffee Shop   
26         North York             4.0                 Coffee Shop   
27         North York             4.0                 Coffee Shop   
28         North York             4.0                 Coffee Shop   
29         North York             4.0                 Coffee Shop   
30         North York             4.0               Grocery Store   
31         North York             4.0               Grocery Store   
32         North York             4.0               Grocery Store   
33         North York             4.0               Grocery Store   
34         North York             4.0                Hockey Arena   
35          East York             4.0                 Pizza Place   
36          East York             4.0                        Park   
37       East Toronto             4.0                 Coffee Shop   
38          East York             4.0                 Coffee Shop   
39          East York             4.0           Indian Restaurant   
41       East Toronto             4.0            Greek Restaurant   
42       East Toronto             4.0                        Park   
43       East Toronto             4.0                        Café   
45    Central Toronto             4.0                         Gym   
46    Central Toronto             4.0              Clothing Store   
47    Central Toronto             4.0              Sandwich Place   
49    Central Toronto             4.0                 Coffee Shop   
51   Downtown Toronto             4.0                 Coffee Shop   
52   Downtown Toronto             4.0                 Coffee Shop   
53   Downtown Toronto             4.0                 Coffee Shop   
54   Downtown Toronto             4.0              Clothing Store   
55   Downtown Toronto             4.0                 Coffee Shop   
56   Downtown Toronto             4.0                 Coffee Shop   
57   Downtown Toronto             4.0                 Coffee Shop   
58   Downtown Toronto             4.0                 Coffee Shop   
59   Downtown Toronto             4.0                 Coffee Shop   
60   Downtown Toronto             4.0                 Coffee Shop   
61   Downtown Toronto             4.0                 Coffee Shop   
62         North York             4.0                  Restaurant   
65    Central Toronto             4.0              Sandwich Place   
66   Downtown Toronto             4.0                        Café   
67   Downtown Toronto             4.0                        Café   
68   Downtown Toronto             4.0             Airport Service   
69   Downtown Toronto             4.0                 Coffee Shop   
70   Downtow

### Cluster 5

In [78]:
tor_merged.loc[tor_merged['Cluster Labels'] == 5, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

Empty DataFrame
Columns: [Borough, Cluster Labels, 1st Most Common Venue, 2nd Most Common Venue, 3rd Most Common Venue, 4th Most Common Venue, 5th Most Common Venue, 6th Most Common Venue, 7th Most Common Venue, 8th Most Common Venue, 9th Most Common Venue, 10th Most Common Venue]
Index: []